In [1]:
from bs4 import BeautifulSoup
import requests
import sqlite3

In [2]:
pmg_url = 'https://www.lds.org/manual/preach-my-gospel-a-guide-to-missionary-service?lang=zho&_r=1'

In [3]:
db = sqlite3.connect('preach_my_gospel_zh.db')
cur = db.cursor()

In [4]:
sql = "DROP TABLE IF EXISTS corpus"
cur.execute(sql)
sql = "CREATE TABLE corpus (url TEXT PRIMARY KEY, type TEXT, html TEXT, string TEXT)"
cur.execute(sql)
sql = 'INSERT OR IGNORE INTO corpus VALUES (?,?,?,?)'

In [5]:
toc_r = requests.get(pmg_url)
soup = BeautifulSoup(toc_r.text, 'html.parser')

In [6]:
pmg_chapter_urls = [a.get('href') for a in soup.find(class_='table-of-content').find_all('a')]
scripture_refs = []
for chapter_url in pmg_chapter_urls:
    chapter_r = requests.get(chapter_url)
    chapter_soup = BeautifulSoup(chapter_r.text, 'html.parser')
    primary = chapter_soup.find(id='primary')
    scripture_refs += [a.get('href') for a in primary.find_all(class_='scripture-ref')]
    cur.execute(sql, (chapter_url, 'preach my gospel', str(primary), primary.get_text()))
db.commit()

In [7]:
scripture_refs = list(set(scripture_refs))
print(len(scripture_refs), 'scripture references')

890 scripture references


In [8]:
for ref_url in scripture_refs:
    if ref_url[0] == "/":
        ref_url = "https://www.lds.org" + ref_url
    ref_r = requests.get(ref_url)
    ref_soup = BeautifulSoup(ref_r.text, 'html.parser')
    highlights = ref_soup.find_all(class_='verse highlight')
    cur.execute(sql, (ref_url, 'scripture', '\n'.join([str(x) for x in highlights]), '\n'.join([x.get_text() for x in highlights])))
db.commit()